## Modeling of Complex Network Project

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import os
data_dir = os.path.expanduser("./data/cora")

In [ ]:
edgelist = pd.read_csv(os.path.join(data_dir, "cora.cites"), sep='\t', header=None, names=["target", "source"])
edgelist["label"] = "cites"
G = nx.from_pandas_edgelist(edgelist, edge_attr="label")
nx.set_node_attributes(G, "paper", "label")

In [ ]:
feature_names = ["w_{}".format(ii) for ii in range(1433)]
column_names =  feature_names + ["subject"]
node_data = pd.read_csv(os.path.join(data_dir, "cora.content"), sep='\t', header=None, names=column_names)

In [ ]:
node_data.head()

In [ ]:
G = nx.read_edgelist(data_dir+"/cora_graph.txt", delimiter=" ", nodetype=int)

### 1. Calculate Node Degree Distribution

In [ ]:
G.degree()
average_degree = nx.average_neighbor_degree(G)
average_degree

In [ ]:
degrees = [degree for _, degree in G.degree()]
print(np.mean(degrees))
# 创建直方图
fig, ax = plt.subplots(figsize=(10, 6))
plt.hist(degrees, bins=range(1, max(degrees)+2), align='left', rwidth=0.8, color='skyblue', edgecolor='darkblue')


plt.title('Degree Distribution', fontsize=16)
plt.xlabel('Degree', fontsize=14)
plt.ylabel('Number of nodes', fontsize=14)


ax.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.6)


plt.tight_layout()
plt.show()


In [ ]:
# 设置seaborn样式
sns.set_style("whitegrid")

# 创建直方图
fig, ax = plt.subplots(figsize=(10, 6))
bins = 2**np.arange(0, np.log2(max(degrees)) + 1)
plt.hist(degrees, bins=bins, align='mid', rwidth=0.97, color='skyblue', edgecolor='black')

# 使用对数-对数轴
ax.set_xscale('log')
ax.set_yscale('log')

# 设置图形标题和标签
plt.title('Degree Distribution', fontsize=16)
plt.xlabel('Degree', fontsize=14)
plt.ylabel('Number of nodes', fontsize=14)

# 调整边距
plt.tight_layout()

# 显示图形
plt.show()

### 2. Calculate Average Shortest Path Length

In [ ]:
shortest_path = dict(nx.all_pairs_dijkstra_path_length(G))
shortest_path_list = []
for key in shortest_path.keys():
    for key2 in shortest_path[key].keys():
        if key == key2:
            continue
        shortest_path_list.append(shortest_path[key][key2])

In [ ]:
np.mean(shortest_path_list)

### 3. Calculate Clustering Coefficient

In [ ]:
clustering_coeffs = nx.clustering(G)
avg_clustering = nx.average_clustering(G)
print(f"Average clustering coefficient: {avg_clustering}")

In [ ]:
for node, coeff in clustering_coeffs.items():
    print(f"Node: {node}, Clustering Coefficient: {coeff}")

### 4. Attack

In [ ]:
initial_lcc = max(nx.connected_components(G), key=len)

In [ ]:
bc = nx.betweenness_centrality(G)

In [ ]:
sorted(list(G.nodes()), key=lambda n: bc[n], reverse=True)

In [ ]:
G.remove_node

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

def attack_network(G, attack_type='random'):
    if attack_type == 'random':
        nodes_to_remove = list(G.nodes())
        np.random.shuffle(nodes_to_remove)
    elif attack_type == 'intentional':
        nodes_to_remove = sorted(G.nodes(), key=lambda n: G.degree(n), reverse=True)
    elif attack_type == 'betweenness':
        bc = nx.betweenness_centrality(G)
        nodes_to_remove = sorted(list(G.nodes()), key=lambda n: bc[n], reverse=True)
    elif attack_type == 'closeness':
        cc = nx.closeness_centrality(G)
        nodes_to_remove = sorted(list(G.nodes()), key=lambda n: cc[n], reverse=True)
    elif attack_type == 'eigenvector':
        ec = nx.eigenvector_centrality(G)
        nodes_to_remove = sorted(list(G.nodes()), key=lambda n: ec[n], reverse=True)
    else:
        raise ValueError("Invalid attack_type")
    
    lcc_sizes = []
    for count, node in enumerate(nodes_to_remove, 1):
        G.remove_node(node)
        lcc_size = len(max(nx.connected_components(G), key=len)) if G else 0
        lcc_sizes.append(lcc_size)
        if lcc_size == 0:
            break
    
    return lcc_sizes

# Attack
print('random')
lcc_random = attack_network(G.copy(), 'random')
print('intentional')
lcc_intentional = attack_network(G.copy(), 'intentional')
print('betweenness')
lcc_betweenness = attack_network(G.copy(), 'betweenness')
print('closeness')
lcc_closeness = attack_network(G.copy(), 'closeness')
print('eigenvector')
lcc_eigenvector = attack_network(G.copy(), 'eigenvector')

# Plot results
plt.figure(figsize=(12, 6))
plt.plot(lcc_random, label='Random Attack', color='blue')
plt.plot(lcc_intentional, label='Intentional Attack (Degree)', color='red')
plt.plot(lcc_betweenness, label='Betweenness Centrality Attack', color='green')
plt.plot(lcc_closeness, label='Closeness Centrality Attack', color='purple')
plt.plot(lcc_eigenvector, label='Eigenvector Centrality Attack', color='orange')
plt.xlabel('Number of Nodes Removed')
plt.ylabel('Size of Largest Connected Component')
plt.legend()
plt.title('Network Robustness under Different Attacks')
plt.show()

In [ ]:
plt.figure(figsize=(12, 7))

# 定义线条样式和颜色
line_styles = ['-', '--', '-.', ':', '-']
colors = ['blue', 'red', 'green', 'purple', 'orange']

# 画图
plt.plot(lcc_random, label='Random Attack', linestyle=line_styles[0], color=colors[0], linewidth=2)
plt.plot(lcc_intentional, label='Intentional Attack (Degree)', linestyle=line_styles[1], color=colors[1], linewidth=2)
plt.plot(lcc_betweenness, label='Betweenness Centrality Attack', linestyle=line_styles[2], color=colors[2], linewidth=2)
plt.plot(lcc_closeness, label='Closeness Centrality Attack', linestyle=line_styles[3], color=colors[3], linewidth=2)
plt.plot(lcc_eigenvector, label='Eigenvector Centrality Attack', linestyle=line_styles[4], color=colors[4], linewidth=2)

# 添加标题和轴标签
plt.title('Network Robustness under Different Attacks', fontsize=16, fontweight='bold')
plt.xlabel('Number of Nodes Removed', fontsize=14)
plt.ylabel('Size of Largest Connected Component', fontsize=14)

# 添加网格
plt.grid(True, linestyle='--', alpha=0.7)

# 改善图例
plt.legend(loc='upper right', fontsize=12, frameon=True, framealpha=0.9)

# 显示图表
plt.show()

In [ ]:
len(G.nodes())

In [ ]:
total_nodes = len(G.nodes())  # 计算总节点数

lcc_random = [size / total_nodes for size in attack_network(G.copy(), 'random')]
lcc_intentional = [size / total_nodes for size in attack_network(G.copy(), 'intentional')]
lcc_betweenness = [size / total_nodes for size in attack_network(G.copy(), 'betweenness')]
lcc_closeness = [size / total_nodes for size in attack_network(G.copy(), 'closeness')]
lcc_eigenvector = [size / total_nodes for size in attack_network(G.copy(), 'eigenvector')]

# 开始绘图
plt.figure(figsize=(12, 8), dpi=300)

# 定义线条样式和颜色
line_styles = ['-', '--', '-.', ':', '-']
colors = ['blue', 'red', 'green', 'purple', 'orange']

# 绘制图形
plt.plot(lcc_random, label='Random Attack', linestyle=line_styles[0], color=colors[0], linewidth=2)
plt.plot(lcc_intentional, label='Intentional Attack (Degree)', linestyle=line_styles[1], color=colors[1], linewidth=2)
plt.plot(lcc_betweenness, label='Betweenness Centrality Attack', linestyle=line_styles[2], color=colors[2], linewidth=2)
plt.plot(lcc_closeness, label='Closeness Centrality Attack', linestyle=line_styles[3], color=colors[3], linewidth=2)
plt.plot(lcc_eigenvector, label='Eigenvector Centrality Attack', linestyle=line_styles[4], color=colors[4], linewidth=2)

# 添加标题和轴标签
plt.title('Analysis of Attacks', fontsize=18, fontweight='bold')
plt.xlabel('Number of Nodes Removed', fontsize=12)
plt.ylabel('Relative Size of Largest Connected Component', fontsize=12)

# 添加网格
plt.grid(True, linestyle='--', alpha=0.7)

# 改善图例
plt.legend(loc='upper right', fontsize=12, frameon=True, framealpha=0.9)

# 显示图表
plt.show()